### Importing the libraries and loading data

In [1]:
!pip install html2text
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import html2text
import re
from string import punctuation
import torch
import pandas as pd
from time import time
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
nltk.download("")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Error loading : Package '' not found in index


False

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
tf.test.gpu_device_name()

''

In [4]:
fulldf = pd.read_csv("drive/MyDrive/Literature Review/question_master.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
fulldf['question_type'].value_counts()

Single Choice Question      179291
Subjective Question          91496
Integer Question              3797
Multiple Choice Question      1483
Paragraph Question             342
Marks the words                 17
Numeric Question                10
Fill in the blanks               7
Match the matrix                 2
Interactive Question             1
Single choice Question           1
Name: question_type, dtype: int64

In [6]:
# df = fulldf.sample(frac = 1)

In [7]:
df = fulldf

In [8]:
# df = df[:35000]

In [9]:
df = df.drop("_class", axis = 1)
df = df.drop("default_difficulty_level_id", axis = 1)
df = df.drop("default_marks", axis = 1)
df = df.drop("forcelly_update", axis = 1)
df = df.drop("is_migrate", axis = 1)
df = df.drop("migrated_date", axis = 1)



In [10]:
df.head()

,_id,objective_type_id,old_stage,question_data,question_id,question_keywords,question_template_id,question_type,question_type_id,question_type_id_original,repository_service_id,repository_syllabus_id,school_id,sort_order,stage,status,user_type,version
0,625d0504f458742102ccd40f,NaN,NaN,"[{""language_id"":1,""language_name"":""English"",""q...",2073394,[],73.0,Subjective Question,19083,NaN,[49060],[698013],17250,4,4,1,3,V001
1,625d0504f458742102ccd418,0.0,1.0,"[{""language_id"":1,""language_name"":""English"",""q...",2073385,[],73.0,Subjective Question,19083,1.0,[49060],[698013],17250,4,1,1,3,V001
2,625d0504f458742102ccd424,0.0,1.0,"[{""language_id"":1,""language_name"":""English"",""q...",2073373,[],73.0,Subjective Question,19083,1.0,[49060],[697895],17250,4,1,1,3,V001
3,625d0504f458742102ccd42b,0.0,1.0,"[{""language_id"":1,""language_name"":""English"",""q...",2073366,[],73.0,Subjective Question,19083,1.0,[49060],[697895],17250,4,1,1,3,V001
4,625d0504f458742102ccd431,0.0,1.0,"[{""language_id"":1,""language_name"":""English"",""q...",2073360,[],73.0,Subjective Question,19083,1.0,[49060],[698013],17250,4,1,1,3,V001


In [11]:
print("Len before:", len(df))
df = df[df['question_data'].notna()]
print("Len after:", len(df))

Len before: 276447
Len after: 276447


In [12]:
#str_to_dict(
df[df['question_type'] == "Multiple Choice Question"].iloc[0]#['question_data'])

_id                                                   625d0504f458742102ccd6a0
objective_type_id                                                         23.0
old_stage                                                                  1.0
question_data                [{"language_id":1,"language_name":"English","q...
question_id                                                            2072737
question_keywords                                                           []
question_template_id                                                      73.0
question_type                                         Multiple Choice Question
question_type_id                                                         19081
question_type_id_original                                                  2.0
repository_service_id                                                  [49059]
repository_syllabus_id                                                [697895]
school_id                                           

In [13]:
def findCorrectAnswerForMCQ(qData):
  ans = ''
  for i in qData['question_option']:
    if i['is_right'] == 1:
      ans = i['answer_text']
  return ans

In [14]:
def findMultipleAnswers(qData):
  ans = ''
  for i in qData['question_option']:
    if i['is_right'] == 1:
      ans += ('. ' + i['answer_text'])
  return ans

In [15]:
def str_to_dict(s):
  false = 1
  true = 1
  return eval(s[1:len(s)-1])

In [16]:
t = time()
texts = []
syllabus_ids = []
answers = []
question_types = []
question_ids = []
question_ids_seen = {}
paragraph_questions_skipped = 0

unknown_syllabus = 0;
for index in range(len(df['question_data'])):
  
  i = df.iloc[index]

  if i['question_id'] in question_ids_seen:
    paragraph_questions_skipped+=1
    continue
  question_ids_seen[i['question_id']] = True
  question_ids.append(i['question_id'])

  if index % 40000 == 0 and index > 0:
    print("Index:",index)

  if i['question_type'] in ["Paragraph Question", "Marks the words", "Numeric Question", "Fill in the blanks", "Match the matrix", "Interactive Question", "Single choice Question"]:
    paragraph_questions_skipped += 1
    continue
  
  text = str_to_dict(i['question_data'])
  syllabus_id = 0
  try:
    text = text['question_text']
  except:
    text = text[0]['question_text']
  
  try:
      syllabus_id = str_to_dict(i['repository_syllabus_id'])
  except:
    syllabus_id = -1
    unknown_syllabus+=1
  texts.append(text)
  syllabus_ids.append(syllabus_id)

  if i['question_type'] == 'Single Choice Question' or i['question_type'] == "Subjective Question":
    try:
      ans = findCorrectAnswerForMCQ(str_to_dict(df.iloc[index]['question_data']))
      answers.append(ans)
    except:
      try:
        ans = findCorrectAnswerForMCQ(str_to_dict(df.iloc[index]['question_data'])[0])
        answers.append(ans)
      except:
        print(index)
  elif i['question_type'] == 'Integer Question':
    answers.append("INTEGER_TYPE_ANSWER")
  elif i['question_type'] == "Multiple Choice Question":
    try:
      ans = findMultipleAnswers(str_to_dict(df.iloc[index]['question_data']))
      answers.append(ans)
    except:
      try:
        ans = findMultipleAnswers(str_to_dict(df.iloc[index]['question_data'])[0])
        answers.append(ans)
      except:
        print(index)

  question_types.append(i['question_type'])
  
                      
print("Time taken: " , int((time() - t)%60) ,"s, ", int((time() - t)/60), "m")
print("Syllabus ID unknown for ", unknown_syllabus, " questions")
print("Total questions skipped (paragraph type, duplicate ids, fill-in-the-blanks, match-the-matrix etc.): ", paragraph_questions_skipped)

Index: 40000
Index: 80000
Index: 120000
Index: 160000
Index: 200000
Index: 240000
Time taken:  46 s,  3 m
Syllabus ID unknown for  214  questions
Total questions skipped (paragraph type, duplicate ids, fill-in-the-blanks, match-the-matrix etc.):  385


In [17]:
def formatToString(q):
  punctuation_marks= []
  tags = ["\n", "\t", " "]
  q = (html2text.html2text(q))
  res=[]
  split = re.findall(r"[\w']+", q)
  for word in split:
    if(word not in tags and word not in punctuation_marks):
      res.append(word)
  mystr = ""
  for i in res:
    mystr=mystr+i+" "
  return mystr

The next cell takes about 5 mins to run

In [18]:
t = time()
textsAsString = [formatToString(i) for i in texts]
answersAsString = [formatToString(i) for i in answers]

print("Time taken: " , int((time() - t)%60) ,"s, ", int((time() - t)/60), "m")
print("Texts:\n",textsAsString[0],"\n",textsAsString[1],"\n And so on...")

Time taken:  22 s,  5 m
Texts:
 Total number of species that can be oxidized by acidic permanganate ion MnO4 H I Fe2 CO2 C2O42 S2 SO32 NO2 PO43 SO42  
 How many following Ammonium salts will evolve N2 gas on heating NH4 2CO3 NH4 2Cr2O7 NH4NO2 NH4ClO4 NH4Cl NH4 2S NH4 2C2O4  
 And so on...


In [19]:
data = pd.DataFrame(list(zip(question_ids, textsAsString, syllabus_ids, answersAsString,question_types )), columns = ["question_id","question", "syllabus_id", "answer", "question_type"])

In [20]:
data.to_csv("drive/MyDrive/Literature Review/Dataset versions/EM-Data-with-answers.csv")

In [21]:
data

,question_id,question,syllabus_id,answer,question_type
0,2073394,Total number of species that can be oxidized b...,698013,All the species which are not in their highest...,Subjective Question
1,2073385,How many following Ammonium salts will evolve ...,698013,Ammonium salts having anions NO2 NO3 ClO4 and ...,Subjective Question
2,2073373,There are some species given below i O2 ii CO ...,697895,4 Molecular orbital configuration for upto 14 ...,Subjective Question
3,2073366,Consider the following compounds 1 H3CF 2 H2CF...,697895,5 stryde images 2022 04 11 2022041195766400164...,Subjective Question
4,2073360,Find total number of reagents which can produc...,698013,Conc H2SO4 2KI K2SO4 HI 12I2 Hg NO3 2 2KI HgI2...,Subjective Question
...,...,...,...,...,...
276057,420544,A dice is rolled thrice The probability of get...,-1,frac 5 24,Single Choice Question
276058,420543,subjective english question,-1,answer_subjective,Subjective Question
276059,415580,question subjective english,-1,answer subjective hindi,Subjective Question
276060,415579,subjective english questions,-1,answer of this question,Subjective Question


In [22]:
data[data['question_type'] == "Multiple Choice Question"]

,question_id,question,syllabus_id,answer,question_type
328,2072737,Incorrect order of bond angle is,697895,XeO4 ClO4 SO42 CF4 NSiH33 OSiH32 OMe2,Multiple Choice Question
339,2072725,H3PO3 ΔH3PO4 PH3,698013,for disproportionation reaction for either the...,Multiple Choice Question
351,2072712,P4 10Cl2 ΔPCl5,698013,for either intermolecular redox reaction or di...,Multiple Choice Question
354,2072709,P4 6Cl2 ΔPCl3,698013,for either intermolecular redox reaction or di...,Multiple Choice Question
360,2072702,NH4NO3 ΔN2O H2O,698013,for comproportionation reaction for either the...,Multiple Choice Question
...,...,...,...,...,...
276019,415623,क छ प रबल समर थक न इत ह स म उनक न म स पष ट क य...,-1,answer option 1 answer option 2 answer option 3,Multiple Choice Question
276031,415609,A dice is rolled thrice The probability of get...,-1,frac 5 24 frac 5 24,Multiple Choice Question
276032,415608,A basket contains 20 guavas and 10 pears out o...,-1,frac 119 435 frac 5 24,Multiple Choice Question
276036,415584,A dice is rolled thrice The probability of get...,-1,524 frac 5 24 245 524 frac 5 24 245 524 frac 5...,Multiple Choice Question
